my_strategy() v2 with:
- earlier approach
- simple 1-move look-ahead recursion

Typically c. 1% losses in 15 seconds

In [1]:
# CONNECT 4 - v2 - SIMPLE LOOKAHEAD

import numpy as np
import time

# Sebastian Lague2
# Debug output
print_debug_output = False

# Globals - board characters
blank = "\u2022 "
red = "\U0001F534"
yellow = "\U0001F7E1"
black = "\u26AB" # black just fills the space for testing
board_chr=[blank, red, yellow, black]

# To interface into tester
def my_strategy(board, player_num):
    # convert list of lists [[int]*6]*7 into np array
    board = np.array(board)
    column, game_finished = get_move(board, player_num)
    return column

# Returns True 50% of the time
def flip_coin():
    return np.random.randint(2) == 1

def is_in_range(x, max):
    return x >= 0 and x < max

def print_debug(str):
    if print_debug_output:
        print(str)
    
def display(board):
    print()
    for i in range(6):
        for j in range(7):
            print(board_chr[board[j,5-i]], end=" ")
        print()     
    print()

def valid_move(board, column):
    if is_in_range(column,7):
        return board[column,5]==0
    return False

def board_is_full(board):
    for column in range(7):
        if board[column,5]==0:
            return False
    return True
   
def do_move(board, player_num, column):
    new_board = board.copy()
    for row in range(6): # deliberately 1 more than allowed to throw an error if column is full
        try:
            if new_board[column,row]==0:
                new_board[column,row]=player_num
                break
        except:
            raise RuntimeError("Illegal move!")
    return new_board, row

def eval_get_counts(board, player_num, column, row, column_step, row_step):
    count_same_adj = 0
    count_same_sep = 0
    count_blank = 0

    # evaluate in both directions along the line dir -1 and 1
    for eval_dir in range(-1,2,2):
        check_column = column + column_step * eval_dir
        check_row = row + row_step * eval_dir
        check_adjacent = True
        # explore up to 3 spaces either side
        for depth in range(3):
            # stop if off the board
            if not (is_in_range(check_column,7) and is_in_range(check_row,6)):
                break
            # stop if square is opponent's
            if board[check_column,check_row]==3-player_num:
                break
            # if the square is a blank, count it but set adjacent to false
            if board[check_column,check_row]==0:
                count_blank += 1
                check_adjacent = False
            # if the square is the same colour, count adjacent or separate
            if board[check_column,check_row]==player_num:
                if check_adjacent:
                    count_same_adj += 1
                else:
                    count_same_sep += 1
            check_column += column_step * eval_dir
            check_row += row_step * eval_dir
    return count_same_adj, count_same_sep, count_blank

def eval_line(board, player_num, column, row, column_step, row_step):
# Returns the overall positional score (or none to force the move), True if game finished

    # eval parameters
    score_same_adj = 2 # exponential
    score_same_sep = 1 # linear
    score_blank = 1 # linear
    eval_line_score = 0

    # Calculate offensive score
    count_same_adj, count_same_sep, count_blank = eval_get_counts(board, player_num, column, row, column_step, row_step)
   
   # If this is a winning move return immediately
    if count_same_adj == 3:
        print_debug("FORCE MOVE - WIN")
        return None, True
    
    # Only score a line if there is enough space to win
    # NB function might return eval_line_score=0 (better than losing move)
    if (count_same_adj + count_same_sep + count_blank) > 2:
        if count_same_adj > 0:
            eval_line_score += score_same_adj ** count_same_adj
        eval_line_score += count_same_sep * score_same_sep
        eval_line_score += count_blank * score_blank
        print_debug(f"offense: count_same_adj: {count_same_adj}, count_same_sep: {count_same_sep}, count_blank: {count_blank}, eval score: {eval_line_score}")

    # Add a defensive score
    count_same_adj, count_same_sep, count_blank = eval_get_counts(board, 3-player_num, column, row, column_step, row_step)
    
    # If this blocks opponent's winning move return immediately
    if count_same_adj == 3:
        print_debug("FORCE MOVE - BLOCK")
        return None, False
    
    # Only score a line if there is enough space for opponent to win
    if (count_same_adj + count_same_sep + count_blank) > 2:
        if count_same_adj > 0:
            eval_line_score += score_same_adj ** count_same_adj
        eval_line_score += count_same_sep * score_same_sep
        eval_line_score += count_blank * score_blank
        print_debug(f"defense: count_same_adj: {count_same_adj}, count_same_sep: {count_same_sep}, count_blank: {count_blank}, eval score: {eval_line_score}")

    return eval_line_score, False, False

def eval_move(board, player, column, row):
    eval_score = 0
    eval_finished = False
    eval_steps = np.array([[1, 0], [1, 1], [0, 1], [1, -1]])
    for i in range(4):
        print_debug(f"evaluating line {i}")
        eval_result = eval_line(board, player, column, row, eval_steps[i,0], eval_steps[i,1])
        # Stop checking other lines if move is forced
        if eval_result[0] == None:
            # Game finished if the move is a winning move
            eval_score = None
            eval_finished = eval_result[1]
            break
        # Otherwise keep adding line scores
        eval_score += eval_result[0]
        
    print_debug(f"column {column} score: {eval_score}")
    return eval_score, eval_finished

def get_move(board, player_num, depth=0):
    best_col = None
    best_eval = -1
    for column in range(7):
        test_losing_move = False
        test_board=board.copy()
        # Check each column from the bottom up
        for row in range(6):
                # Find first available space in the column (if any)...
                if board[column, row] == 0:
                    print_debug(f"\n------\nColumn: {column}")
                    test_board[column,row] = player_num

                    # First, look ahead if this is the proposed move
                    if depth == 0:
                        look_ahead_board = test_board.copy()
                        look_ahead_result = get_move(look_ahead_board, 3-player_num, depth+1)
                        test_losing_move = look_ahead_result[1]
                        
                    # Second, evaluate the move
                    test_result = eval_move(test_board, player_num, column, row)
                    test_finished = test_result[1]
                    test_eval = test_result[0]
                    # If the move is forced, return the best column and whether the game is finished
                    if test_eval == None:
                        return column, test_finished
                    # Otherwise, if this is a losing move...
                    if test_losing_move:
                        # If this is the only move so far then grudgingly take it for now
                        # Leave best_eval = -1
                        if best_col == None:
                            best_col = column
                            print_debug("Taking this losing move for now")
                        # Don't bother evaluating a losing move
                        break

                    # Otherwise, update the best column if this is the best move so far
                    if (test_eval > best_eval) or (test_eval == best_eval and flip_coin()):
                        print_debug("found better / the same and flipped a coin")
                        best_eval = test_eval
                        best_col = column
                    # Don't check further rows in this column
                    break

    # If no valid moves left, raise an error (this should never happen)
    if best_col == None:
        raise RuntimeError("Board is full, no valid moves left!")
        return None
    
    # return the best column and game finished is False
    return best_col, False

In [2]:
# When you're ready to run your strategy run the top cell, then this cell
# You can do this as often as you like as you improve your strategy
from assessment.assessor import assess

assess(my_strategy, 1000)

Assessing student strategy...
                                                                                                    
Results
Wins: 995
Draws: 0
Losses: 5
Forfeits: 0
Mark: 98.88%
